In [1]:
# 计算语料库的平均长度

file_path = 'data/wiki1m_for_simcse.txt'

total_length = 0
total_sentences = 0

with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        words = line.strip().split()
        total_length += len(words)
        total_sentences += 1

average_length = total_length / total_sentences
print(average_length)


19.432477


In [ ]:
# 将语料库分片
import os
import shutil

file_path = 'data/wiki1m_for_simcse.txt'
output_dir = 'data/wiki1m_for_simcse_splited'
chunks = 10

if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.mkdir(output_dir)

with open(file_path, 'r', encoding='utf-8') as file:
    for i, line in enumerate(file):
        output_file_path = os.path.join(output_dir, str(i % chunks) + '.txt')
        with open(output_file_path, 'a', encoding='utf-8') as output_file:
            output_file.write(line)

In [ ]:
# es
from elasticsearch import Elasticsearch

es = Elasticsearch(hosts="http://59.77.134.205:9200")

index_name = "wikidata-latest"

query = {"query": {"match": {"content": "China"}}}
result = es.search(index=index_name, body=query)
print(result)



## spacy记录

不同模型有不同精度

#### [en_core_web_sm](https://spacy.io/models/en#en_core_web_sm)

支持的实体类型

<img src="https://lyu-oss.oss-cn-beijing.aliyuncs.com/img-bed/image-20231227161004504.png" alt="image-20231227161004504" style="zoom:50%;" />

* CARDINAL -- 数字 【'九百多', '8000', '八百'，'1111.01'】
* DATE -- 大粒度时间，时间段 【 '今年', '明天', '今天', '国庆期间', '3天', '10天'， '三年前'】
* **EVENT -- 事件 【'伦敦奥运会', '世界杯','第14届中国国际工业博览会', '深圳市五届人大二次会议'】**
* **FAC -- 小地点 【'轻轨1号线锡北运河站', '万达广场', '乐购超市'，'永盛大酒店', '110岗亭'】**
* **GPE -- 地点 【'美国', '加拿大', '北京', '中国',】**
* **LANGUAGE -- 语言 【'英语', '汉语', '上海话', '中文'】**
* **LAW -- 规章制度 【'青少年犯罪法', '阿鲁巴决议', '反托拉斯法'】**
* **LOC -- 大地点 【'欧洲人', '欧洲', '亚洲', '天山山脉', '巴尔斯卡乌尼河''月球', '火星'】**
* MONEY -- 货币 【'￥8000', '9200', '60元', '3000美金'】部分省略货币单位也能识别
* **NORP -- 人物、地点 【'德国', '中国', '中国人','韩版', '日媒', '日本', '扬州'】**
* ORDINAL -- 顺序 【'首', '第六', '第二', '第一','第几'】
* **ORG -- 组织 【'杭州江干区公安分局'，'LG', '三星', '苹果'，'中国移动', '央行'，'中央社'， '外交部'】**
* PERCENT -- 比率 【'0.2%', '0.2%', '48%'】
* **PERSON -- 人物 【'栾丽娜', '斯蒂芬·弗雷斯', '栗元广', '小雨', '小银狐'】**
* **PRODUCT -- 产品 【'Android', 'iOS'， '金龙鱼', 'UCWeb的浏览器'】**
* QUANTITY -- 量级 【'87,000', '46.522吨', '48248.8千克', '0.23点'，'5.3级', '4200公里' 】
* TIME -- 时间 【'十分钟', '下午', '七点', '今晚'】
* **WORK_OF_ART -- 艺术品 【'刺客信条', '富春山居图', '有一说一'，'探索•发现', '定窑考工记'】**

事件、地点、语言、规章制度、人物、组织、产品、艺术品

In [ ]:
#分词统计
# 统计语料库中各种类型的实体数量

# 事件、地点、语言、规章制度、人物、组织、产品、艺术品
import spacy
from tqdm import tqdm
from collections import Counter

type_list = ['EVENT', 'FAC', 'GPE', 'LANGUAGE', 'LAW', 'LOC', 'NORP', 'ORG', 'PERSON', 'PRODUCT', 'WORK_OF_ART']

nlp = spacy.load("en_core_web_sm")

file_path = 'data/wiki1m_for_simcse.txt'

type_counts = Counter()

with open(file_path, 'r', encoding='utf-8') as file:
    for index, line in tqdm(enumerate(file)):
        words = line.strip().split()
        text = ' '.join(words)
        doc = nlp(text)
        for ent in doc.ents:
            if ent.label_ in type_list:
                type_counts[ent.label_] += 1
    

for type_name, count in type_counts.items():
    print(f"{type_name}: {count}")
# GPE: 373869
# NORP: 152530
# ORG: 532072
# WORK_OF_ART: 80636
# FAC: 37106
# PERSON: 505931
# LOC: 56057
# PRODUCT: 22275
# LAW: 6742
# EVENT: 29668
# LANGUAGE: 8100
# sum: 1,763,006
    
# 事件:P31: Q1656682
# 地点:P31: Q2221906
# 语言:P31: Q34770
# 规章制度:P31: Q22097341
# 人物:P31: Q5
# 组织:P31: Q43229
# 产品:P31: Q2424752
# 艺术品:P31: Q838948

In [ ]:
# es检索测试
from elasticsearch import Elasticsearch
es = Elasticsearch(hosts="http://59.77.134.205:9200")
index_name = "wikidata-filter"


# 执行精确匹配查询
exact_match_query = {
    "query": {
        "match": {
            "enwiki_title.keyword": {
                "query": "Taylor Swift",
                "operator": "and"
            }
        }
    },
    "_source": ["description"]

}

result = es.search(index=index_name, body=exact_match_query)
print(result['hits']['hits'][0]['_source']['description'])


In [5]:
import spacy
from tqdm import tqdm
from elasticsearch import Elasticsearch
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import ProcessPoolExecutor

spacy_tool = spacy.load("en_core_web_sm")
 
file_path = 'data/wiki1m_for_simcse.txt'
output_path = 'data/wiki1m_for_simcse_person_bracket.txt'
total_lines = 1000000

es = Elasticsearch(hosts="http://59.77.134.205:9200")
index_name = "wikidata-filter"

type_list = ['PERSON','LANGUAGE','ORG']

def search_entity_description(entity_name):
    query = {
        "query": {
            "match": {
                "enwiki_title.keyword": {
                    "query": entity_name,
                    "operator": "and"
                }
            }
        },
        "_source": ["description"]
    }
    result = es.search(index=index_name, body=query)
    if len(result['hits']['hits']) > 0:
        return result['hits']['hits'][0]['_source']['description']
    else:
        return None

def process_line(line):
    doc = spacy_tool(line)
    new_line = line.strip()  # 保留原句
    replaced_entities = set()  # 用集合存储已经替换过的实体
    for ent in doc.ents:
        if ent.label_ in type_list and ent.text not in replaced_entities:
            description = search_entity_description(ent.text)
            if description:
                new_line = new_line.replace(ent.text, f"{ent.text} ({description})")
                replaced_entities.add(ent.text)  # 添加已替换的实体到集合中
    return new_line

count = 0
with open(file_path, 'r', encoding='utf-8') as file, open(output_path, 'w', encoding='utf-8') as output_file:
    with ProcessPoolExecutor() as executor:
        for result_line in tqdm(executor.map(process_line, file), total=total_lines):
            output_file.write(result_line + "\n")


100%|██████████| 1000000/1000000 [3:39:52<00:00, 75.80it/s]    


In [ ]:
# 将数据插入es
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from qwikidata.json_dump import WikidataJsonDump
from qwikidata.entity import WikidataItem
from qwikidata.utils import dump_entities_to_json
import json
import os
import time
from tqdm import tqdm

es = Elasticsearch(hosts="http://localhost:9200")
index_name = "wikidata-filter"
es.indices.create(index=index_name,ignore=400)

wjd_dump_path = "wikidata\wikidata-20231215-all.json.bz2"
wjd = WikidataJsonDump(wjd_dump_path)

batch_size=10000
batch_num=0
item_list=[]

property_list = set(['Q1656682','Q2221906','Q34770','Q22097341','Q5','Q43229','Q2424752','Q838948'])

for ii, entity_dict in tqdm(enumerate(wjd)):
    if entity_dict["type"] == "item":
        entity = WikidataItem(entity_dict)
        if ii< 15000000:
            continue
        try:
            # 判断es中是否已经存在
            if es.exists(index=index_name,id=entity.entity_id):
                continue

            # 判断是否存在英文wiki标题
            en_wiki_title = entity.get_enwiki_title()
            if not bool(en_wiki_title):
                continue

            # 判断是否存在英文标签
            en_label = entity.get_label('en')
            if not bool(en_label):
                continue
            # 判断是否存在英文描述
            en_description = entity.get_description('en')
            if not bool(en_description):
                continue
            
            # 判断是否有P31
            instance_list = entity.get_claim_group('P31')
            if len(instance_list) <=0:
                continue
            
            # 判断P31 是否在 list中
            instance_value_list = [claim.mainsnak.datavalue.value["id"] for claim in entity.get_truthy_claim_group('P31')]
            if not bool(property_list & set(instance_value_list)):
                continue
        except Exception as e:
            # 捕获任意异常
            print(e)
            continue

        item={
            '_id': entity.entity_id,
            '_index': index_name,
            'entity_id':entity.entity_id,
            'label':en_label,
            'description':en_description,
            'enwiki_title':en_wiki_title,
            'instance_of': str(instance_value_list)
        }
        item_list.append(item)
        # 每bs条插入
        if len(item_list) > batch_size:
            helpers.bulk(client=es, actions=item_list)
            batch_num+=1
            item_list=[]
            print(f"batch num:{batch_num} has been inserted.The data efficiency rate is {batch_num * batch_size / ii}")

# 插入余下的数据
helpers.bulk(client=es, actions=item_list)


In [3]:
# 构建CSV
import csv
from tqdm import tqdm

def merge_to_csv(file1_path, file2_path, output_csv_path):
    with open(file1_path, 'r', encoding='utf-8') as file1, open(file2_path, 'r', encoding='utf-8') as file2:
        lines1 = file1.readlines()
        lines2 = file2.readlines()
        num_lines = len(lines1)
    

    # Use tqdm for progress visualization
    with tqdm(total=num_lines, desc='Merging to CSV', unit='lines') as pbar:
        # Combine lines from both files into a list of tuples
        combined_lines = [(lines1[i].strip(), lines2[i].strip()) for i in range(num_lines)]
        pbar.update(num_lines)

    # Write the combined data to a CSV file
    with open(output_csv_path, 'w', newline='', encoding='utf-8') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['sent0', 'sent1'])  # Header
        csv_writer.writerows(combined_lines)

sent0_file = 'data/wiki1m_for_simcse.txt'
sent1_file = 'data/wiki1m_for_simcse.txt'
output_file = 'data/wiki1m_for_simcse_self_positive.csv'

merge_to_csv(sent0_file, sent1_file, output_file)


Merging to CSV: 100%|██████████| 1000000/1000000 [00:01<00:00, 760608.62lines/s]


In [1]:
# 构建csv三元组
import csv
from tqdm import tqdm

def merge_to_csv(file1_path, file2_path, file3_path, output_csv_path):
    with open(file1_path, 'r', encoding='utf-8') as file1, open(file2_path, 'r', encoding='utf-8') as file2, open(file3_path, 'r', encoding='utf-8') as file3:
        lines1 = file1.readlines()
        lines2 = file2.readlines()
        lines3 = file3.readlines()
        num_lines = len(lines1)
    

    # Use tqdm for progress visualization
    with tqdm(total=num_lines, desc='Merging to CSV', unit='lines') as pbar:
        # Combine lines from both files into a list of tuples
        combined_lines = [(lines1[i].strip(), lines2[i].strip(), lines3[i].strip()) for i in range(num_lines)]
        pbar.update(num_lines)

    # Write the combined data to a CSV file
    with open(output_csv_path, 'w', newline='', encoding='utf-8') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['sent0', 'sent1', 'sent2'])  # Header
        csv_writer.writerows(combined_lines)

sent0_file = 'data/wiki1m_for_simcse.txt'
sent1_file = 'data/wiki1m_for_simcse.txt'
sent2_file = 'data/wiki1m_for_simcse_person_description.txt'
output_file = 'data/simcse_(origin_origin_person).csv'

merge_to_csv(sent0_file, sent1_file, sent2_file, output_file)


Merging to CSV: 100%|██████████| 1000000/1000000 [00:01<00:00, 766361.89lines/s]


In [3]:
# 随机采样
import random

def sample_sentences(input_file, output_file, num_samples):
    # 读取输入文件的所有句子
    with open(input_file, 'r', encoding='utf-8') as f:
        sentences = f.readlines()

    # 随机采样指定数量的句子
    sampled_sentences = random.sample(sentences, num_samples)

    # 将采样的句子写入输出文件
    with open(output_file, 'w', encoding='utf-8') as f:
        for sentence in sampled_sentences:
            f.write(sentence)

# 示例：从 input_file 中随机采样 1000 个句子到 output_file 中
input_file = 'data/wiki1m_for_simcse.txt'
output_file = 'data/wiki1k_for_debug.txt'
num_samples = 1000
sample_sentences(input_file, output_file, num_samples)